# Imports

In [1]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

# https://towardsdatascience.com/data-science-for-cycling-how-to-read-gpx-strava-routes-with-python-e45714d5da23
import gpxpy
import gpxpy.gpx

# Lesson 5.02
from bs4 import BeautifulSoup
import lxml

# https://stackoverflow.com/questions/54873868/python-calculate-bearing-between-two-lat-long
from geographiclib.geodesic import Geodesic

%matplotlib inline
pd.set_option('display.max_columns', None)

import warnings
warnings.simplefilter("ignore")

# Function - Build Out - gpx_to_df()

## Garmin .gpx File

In [2]:
# Open .gpx file and parse xml
with open('../data/activity_9305828885.gpx', 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

In [3]:
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

In [4]:
# check length of tracks
print(f'Tracks: {len(gpx.tracks)}')
# check number of data points
print(f'No. of Track Points: {gpx.get_track_points_no()}')

Tracks: 1
No. of Track Points: 1333


In [5]:
# explore
moving_data = gpx.get_moving_data(raw=True)
moving_data

MovingData(moving_time=4591.0, stopped_time=601.0, moving_distance=27617.552711962377, stopped_distance=24.779016778381447, max_speed=49.9794491353225)

In [6]:
gpx.get_uphill_downhill()

UphillDownhill(uphill=140.3600003242493, downhill=139.56000108718894)

In [7]:
gpx.extensions

[]

## Create Dataset

In [8]:
# check out .gpx file as xml
# gpx.to_xml()[:2000]

In [9]:
# extract timestamp, lat, lon, and elevation from gpx file
route_info = []

for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            route_info.append({
                'timestamp': point.time,
                'latitude': point.latitude,
                'longitude': point.longitude,
                'elevation': point.elevation,
            })

In [10]:
route_df = pd.DataFrame(route_info)

In [11]:
route_df.head()

,timestamp,latitude,longitude,elevation
0,2022-07-30 19:29:44+00:00,38.773742,-121.363474,35.200001
1,2022-07-30 19:29:45+00:00,38.773660,-121.364041,35.000000
2,2022-07-30 19:29:49+00:00,38.773795,-121.363652,35.000000
3,2022-07-30 19:29:51+00:00,38.773860,-121.363689,35.000000
4,2022-07-30 19:29:53+00:00,38.773917,-121.363619,35.000000


## BeautifulSoup for Extension Data - heartrate, cadence

In [12]:
xml = gpx.to_xml()
soup = BeautifulSoup(xml, features = 'xml')

In [13]:
time_soup = soup.find_all('time')
hr_soup = soup.find_all('ns3:hr')
cad_soup = soup.find_all('ns3:cad')

In [14]:
type(time_soup)

bs4.element.ResultSet

In [15]:
len(time_soup)

1334

In [16]:
type(hr_soup)

bs4.element.ResultSet

In [17]:
len(hr_soup)

1333

In [18]:
len(soup.find_all('ns3:hr')[-len(cad_soup):])

1331

In [19]:
len(cad_soup)

1331

In [20]:
len(route_df)

1333

In [21]:
# chop off early points from index - accounts for before cadence sensor calibration
route_df = route_df[-len(cad_soup):]

# for loop to extract heart rate and cadence values from each point
hr_list = []
cad_list = []
for i in hr_soup:
    hr_list.append(i.get_text(strip = True))
    
for j in cad_soup:
    cad_list.append(j.get_text(strip = True))

In [22]:
# add columns to dataset from extraction
route_df['heart_rate'] = hr_list[-len(cad_soup):]
route_df['cadence'] = cad_list

In [23]:
route_df['cadence'].value_counts()

0     184
65     72
67     63
66     62
64     58
     ... 
95      1
17      1
93      1
12      1
15      1
Name: cadence, Length: 71, dtype: int64

# Function - gpx_to_df() 

In [32]:
def gpx_to_df(filepath):
    
    # Open .gpx file and parse xml
    with open(filepath, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
        
    # check length of tracks
    print(f'Tracks: {len(gpx.tracks)}')
    # check number of data points
    print(f'No. of Track Points: {gpx.get_track_points_no()}')
    
    # extract timestamp, lat, lon, and elevation from gpx file
    route_info = []

    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                route_info.append({
                    'timestamp': point.time,
                    'latitude': point.latitude,
                    'longitude': point.longitude,
                    'elevation': point.elevation,
                })
    
    # create dataframe
    route_df = pd.DataFrame(route_info)
    
    ########################
    # timestamp to datetime
    route_df['timestamp'] = pd.to_datetime(route_df['timestamp'])
    
    # elapsed time
    route_df['time_diff_s'] = 0
    for i in range(1, len(route_df)-1):
        t1 = route_df.iloc[i]['timestamp']
        t2 = route_df.iloc[i-1]['timestamp']
        # https://www.geeksforgeeks.org/how-to-set-cell-value-in-pandas-dataframe/
        route_df.at[i, 'time_diff_s'] = (t1 - t2).seconds
    route_df['total_time_s'] = route_df['time_diff_s'].cumsum(skipna = True)
    
    #######################
    # elevation change
    
    route_df['ele_diff'] = 0
    for i in range(1, len(route_df)-1):
        e1 = route_df.iloc[i]['elevation']
        e2 = route_df.iloc[i-1]['elevation']
        route_df.at[i, 'ele_diff'] = (e1 - e2)
    route_df['total_ele_change'] = route_df['ele_diff'].cumsum(skipna = True)
    
    #######################
    # add extension data with beautifulsoup
    xml = gpx.to_xml()
    soup = BeautifulSoup(xml, features = 'xml')
    
    # Use Beautiful Soup to find all heart rate and cadence
    hr_soup = soup.find_all('ns3:hr')
    cad_soup = soup.find_all('ns3:cad')
    
    # chop off early points from index - accounts for before cadence sensor calibration
    route_df = route_df[-len(cad_soup):]

    # for loop to extract heart rate and cadence values from each point
    hr_list = []
    cad_list = []
    for i in hr_soup:
        # https://stackoverflow.com/questions/69420686/bs4-element-resultset-elements-to-a-list
        hr_list.append(i.get_text(strip = True))

    for j in cad_soup:
        cad_list.append(j.get_text(strip = True))

    # add columns to dataset from extraction
    route_df['heart_rate'] = hr_list[-len(cad_soup):]
    route_df['cadence'] = cad_list
   
    #######################
    route_df.reset_index(drop = True, inplace = True)
    
    return route_df

In [33]:
_885 = gpx_to_df('../data/activity_9305828885.gpx')

Tracks: 1
No. of Track Points: 1333


In [35]:
_885.tail(12)

,timestamp,latitude,longitude,elevation,time_diff_s,total_time_s,ele_diff,total_ele_change,heart_rate,cadence
1319,2022-07-30 20:55:51+00:00,38.774140,-121.362701,35.799999,6,5167,0.399998,0.599998,126,61
1320,2022-07-30 20:55:57+00:00,38.774163,-121.363078,36.000000,6,5173,0.200001,0.799999,129,62
1321,2022-07-30 20:55:59+00:00,38.774184,-121.363198,35.799999,2,5175,-0.200001,0.599998,130,62
1322,2022-07-30 20:56:00+00:00,38.774190,-121.363258,35.799999,1,5176,0.000000,0.599998,130,62
1323,2022-07-30 20:56:02+00:00,38.774178,-121.363369,35.799999,2,5178,0.000000,0.599998,130,0
1324,2022-07-30 20:56:03+00:00,38.774156,-121.363410,35.799999,1,5179,0.000000,0.599998,130,0
1325,2022-07-30 20:56:05+00:00,38.774088,-121.363469,35.799999,2,5181,0.000000,0.599998,130,9
1326,2022-07-30 20:56:07+00:00,38.774021,-121.363510,36.000000,2,5183,0.200001,0.799999,129,15
1327,2022-07-30 20:56:14+00:00,38.773811,-121.363648,36.200001,7,5190,0.200001,1.000000,131,0
1328,2022-07-30 20:56:15+00:00,38.773788,-121.363646,36.200001,1,5191,0.000000,1.000000,128,0


# Create Bearing Column

In [ ]:
# https://stackoverflow.com/questions/54873868/python-calculate-bearing-between-two-lat-long
# https://geographiclib.sourceforge.io/html/python/code.html#geographiclib.geodesic.Geodesic.AZIMUTH
def get_bearing(lat1, lat2, long1, long2):
    brng = Geodesic.WGS84.Inverse(lat1, long1, lat2, long2)['azi1']
    return brng

In [ ]:
get_bearing(38.773795, 38.773860, -121.363652, -121.363689)

In [ ]:
_885['bearing'] = 0
for i in range(1, len(_885)):
    lat1 = _885.latitude.iloc[i-1]
    lat2 = _885.latitude.iloc[i]
    long1 = _885.longitude.iloc[i-1]
    long2 = _885.longitude.iloc[i]
    bearing = get_bearing(lat1, lat2, long1, long2)
    # https://stats.stackexchange.com/questions/283572/using-iloc-to-set-values
    _885.bearing.iloc[[i]] = bearing

In [ ]:
_885.head(10)

# Plot

In [ ]:
# https://www.python-graph-gallery.com/43-use-categorical-variable-to-color-scatterplot-seaborn
# Use the 'hue' argument to provide a factor variable
plt.figure(figsize = (16, 8))
sns.lmplot( x="longitude", y="latitude", data=_885, fit_reg=False, 
           hue='heart_rate', 
           palette = 'viridis', 
           height = 10, aspect = 2)
 
# Move the legend to an empty part of the plot
# plt.legend(loc='lower right')

plt.show();